# Pricing API Client Demo

This notebook shows how quants interact with the Pricing API using the Python client. The client uses **sgqlc** to call the GraphQL API; when pricing and risk operations are added, the same pattern applies.

**Prerequisites:** Run `docker-compose up` from the repo root. The client is pre-installed in this Jupyter environment. The API is available at `http://api:8000/graphql` from inside the Docker network.

In [1]:
from pricing_client import PricingClient

# Default URL targets the API service (hostname 'api') inside Docker Compose
client = PricingClient("http://api:8000/graphql")

In [2]:
result = client.hello("Quant Team")
print(result)

Hello Quant Team from Pricing API!


In [3]:
version = client.version()
print(f"API version: {version}")

API version: 0.1.0


## Shared market inputs

Curves and FX spot used across all product examples below. **USD_DISC** is the discount curve that will be streamed in the real-time section.

In [ ]:
from pricing_client import (
    CurveInput,
    FxSpotInput,
    HazardCurveInput,
    MarketInput,
)

# USD and EUR discount curves; hazard curve for CDS; EURUSD spot for FX forward
usd_curve = CurveInput(
    name="USD_DISC",
    pillars=[0.5, 1.0, 2.0, 5.0, 10.0],
    zero_rates_cc=[0.045, 0.043, 0.040, 0.038, 0.037],
)
eur_curve = CurveInput(
    name="EUR_DISC",
    pillars=[0.5, 1.0, 2.0, 5.0, 10.0],
    zero_rates_cc=[0.040, 0.038, 0.036, 0.034, 0.033],
)
hazard_curve = HazardCurveInput(
    name="CORP_HAZ",
    pillars=[0.5, 1.0, 2.0, 5.0, 10.0],
    hazard_rates=[0.01, 0.01, 0.01, 0.01, 0.01],
)
fx_spot_eurusd = FxSpotInput(pair="EURUSD", spot=1.08)

# Full market: used by all product examples (each product uses the curves it needs)
market_full = MarketInput(
    curves=[usd_curve, eur_curve],
    hazard_curves=[hazard_curve],
    fx_spot=[fx_spot_eurusd],
)

## Zero-coupon bond

Price a 2Y zero-coupon bond with 1M notional; request PV01 (parallel curve bump).

**Whose position:** Bond **holder** (you receive 1M at maturity). **NPV:** Present value of that cashflow (discounted with the USD curve). NPV &gt; 0 is a mark-to-market **gain** for the holder (asset worth more than zero). **PV01:** Change in NPV for a 1 bp rise in the curve; negative PV01 means rates up ⇒ price down ⇒ holder **loses** value when rates rise.

In [ ]:
from pricing_client import ZeroCouponBondInput

bond = ZeroCouponBondInput(curve="USD_DISC", maturity=2.0, notional=1_000_000)
result = client.price_zero_coupon_bond(bond, market_full, calculate_pv01=True)

print(f"NPV:  {result.npv:,.2f}")
print(f"PV01: {result.pv01:,.2f}")

## Fixed-float swap

Price a 2Y fixed-float swap (10M notional, 4% fixed); request PV01.

**Whose position:** Payer of fixed / receiver of float (you pay 4% fixed, receive float). **NPV:** Mark-to-market of the swap from your perspective. Positive NPV = **gain** (swap is in your favour); negative NPV = **loss**. **PV01:** Change in NPV for a 1 bp rise in the curve; positive PV01 means rates up ⇒ float leg worth more ⇒ you **gain** when rates rise; you **lose** when rates fall.

In [ ]:
from pricing_client import FixedFloatSwapInput

swap = FixedFloatSwapInput(
    curve="USD_DISC",
    notional=10_000_000,
    fixed_rate=0.04,
    pay_times=[0.5, 1.0, 1.5, 2.0],
)
result = client.price_swap(swap, market_full, calculate_pv01=True)

print(f"NPV:  {result.npv:,.2f}")
print(f"PV01: {result.pv01:,.2f}")

## FX forward

Price a 1Y FX forward (5M EUR notional, strike 1.085) using CIP; request PV01 and FX delta.

**Whose position:** Long EUR / short USD (you buy EUR at strike 1.085 at maturity). **NPV:** Value in USD of the forward from your perspective. Positive NPV = **gain** (forward is in the money); negative NPV = **loss**. **PV01:** Sensitivity to a 1 bp move in the discount curves. **FX delta:** Sensitivity to a 1% move in EURUSD spot; positive delta means you **gain** if EUR strengthens, **lose** if EUR weakens.

In [ ]:
from pricing_client import FXForwardInput

forward = FXForwardInput(
    pair="EURUSD",
    base_curve="EUR_DISC",
    quote_curve="USD_DISC",
    maturity=1.0,
    notional_base=5_000_000,
    strike=1.085,
)
result = client.price_fx_forward(forward, market_full, calculate_pv01=True, calculate_fx_delta=True)

print(f"NPV:      {result.npv:,.2f}")
print(f"PV01:     {result.pv01:,.2f}")
print(f"FX delta: {result.fx_delta:,.2f}")

## Level-pay mortgage

Price a 10Y level-pay mortgage (1M notional, 4% annual rate, 12 payments/year); request PV01.

**Whose position:** **Lender** (bank): you lent 1M and receive fixed principal + interest. **NPV:** Present value of those cashflows (mark-to-market value of the loan asset). NPV &gt; notional = **gain** (asset worth more than par); NPV &lt; notional = **loss**. **PV01:** Change in NPV for a 1 bp rise in the curve; negative PV01 means rates up ⇒ value down ⇒ lender **loses** when rates rise, **gains** when rates fall (fixed rate).

In [ ]:
from pricing_client import MortgageInput

mortgage = MortgageInput(
    curve="USD_DISC",
    notional=1_000_000,
    annual_rate=0.04,
    term_years=10.0,
    payments_per_year=12,
)
result = client.price_mortgage(mortgage, market_full, calculate_pv01=True)

print(f"NPV:  {result.npv:,.2f}")
print(f"PV01: {result.pv01:,.2f}")

## CDS

Price a 5Y single-name CDS (10M notional, 100bp spread, protection buyer); request CS01 (hazard curve sensitivity).

**Whose position:** **Protection buyer** (you pay premium, receive protection if the reference entity defaults). **NPV:** Value of the CDS from your perspective (PV of protection leg − PV of premium leg). Negative NPV = **loss** (premiums exceed protection value; spread is rich). **CS01:** Change in NPV for a 1 bp rise in the hazard curve; positive CS01 means credit widens ⇒ protection more valuable ⇒ you **gain** when credit widens, **lose** when it tightens.

In [ ]:
from pricing_client import CDSInput

cds = CDSInput(
    discount_curve="USD_DISC",
    survival_curve="CORP_HAZ",
    notional=10_000_000,
    premium_rate=0.01,
    pay_times=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
    recovery=0.4,
)
result = client.price_cds(cds, market_full, calculate_cs01=True)

print(f"NPV:  {result.npv:,.2f}")
print(f"CS01: {result.cs01:,.2f}")

**Interpreting the numbers (this example):**

**Whose position:** Protection **buyer** (you pay 100 bp premium, receive protection if the reference entity defaults). Negative NPV = mark-to-market **loss** (you are paying more than fair value). Positive CS01 = you **gain** when credit widens, **lose** when it tightens.

- **NPV ≈ −171,924 (USD)** — NPV = PV(protection leg) − PV(premium leg). A negative NPV means the premiums you pay (100 bp on 10M notional) are worth more in present value than the protection you receive, given the current hazard curve (1% flat). So the spread is **rich**: you are paying more than fair value for the protection. If the name’s credit were to widen (higher default risk), the protection leg would be worth more and NPV would move toward zero or positive.

- **CS01 ≈ 2,709 (USD per 1 bp)** — Sensitivity of NPV to a 1 bp parallel increase in the hazard curve. Positive CS01 for a protection buyer means: if the reference name’s credit widens by 1 bp (hazard rate up 1 bp), the position’s NPV increases by about 2,709 USD (NPV becomes less negative). So you gain roughly 2,709 USD per 1 bp widening. CS01 scales with notional and maturity; for 10M notional and 5Y, a CS01 in the low thousands is typical.

## Real-time: all products on USD_DISC ticks

Subscribe to **USD_DISC** curve updates; on each tick, reprice every product and refresh the table below. EUR_DISC, CORP_HAZ, and EURUSD spot stay fixed. Run until interrupted (Ctrl+C) or set `max_updates` to limit ticks.

In [ ]:
from IPython.display import display
from pricing_client import (
    CDSInput,
    FXForwardInput,
    FixedFloatSwapInput,
    LiveBlotter,
    MortgageInput,
    ZeroCouponBondInput,
    stream_live_blotter,
)

# Product definitions (same as examples above)
bond = ZeroCouponBondInput(curve="USD_DISC", maturity=2.0, notional=1_000_000)
swap = FixedFloatSwapInput(
    curve="USD_DISC",
    notional=10_000_000,
    fixed_rate=0.04,
    pay_times=[0.5, 1.0, 1.5, 2.0],
)
forward = FXForwardInput(
    pair="EURUSD",
    base_curve="EUR_DISC",
    quote_curve="USD_DISC",
    maturity=1.0,
    notional_base=5_000_000,
    strike=1.085,
)
mortgage = MortgageInput(
    curve="USD_DISC",
    notional=1_000_000,
    annual_rate=0.04,
    term_years=10.0,
    payments_per_year=12,
)
cds = CDSInput(
    discount_curve="USD_DISC",
    survival_curve="CORP_HAZ",
    notional=10_000_000,
    premium_rate=0.01,
    pay_times=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
    recovery=0.4,
)

products = [
    ("Zero-coupon bond", bond, lambda c, m: c.price_zero_coupon_bond(bond, m, calculate_pv01=True)),
    ("Fixed-float swap", swap, lambda c, m: c.price_swap(swap, m, calculate_pv01=True)),
    ("FX forward", forward, lambda c, m: c.price_fx_forward(forward, m, calculate_pv01=True, calculate_fx_delta=True)),
    ("Level-pay mortgage", mortgage, lambda c, m: c.price_mortgage(mortgage, m, calculate_pv01=True)),
    ("CDS", cds, lambda c, m: c.price_cds(cds, m, calculate_cs01=True)),
]

blotter = LiveBlotter(title="Live pricing (USD_DISC)")
display(blotter.widget)

await stream_live_blotter(
    client,
    blotter,
    products,
    static_curves=[eur_curve],
    hazard_curves=[hazard_curve],
    fx_spot=[fx_spot_eurusd],
    live_curve_name="USD_DISC",
    marketdata_url="ws://marketdata:8001/graphql",
    max_updates=None,
)